<a href="https://colab.research.google.com/github/heros-lab/colaboratory/blob/master/verification_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')


In [2]:
def filter(pd_series, labels):
    q1 = pd_series.quantile(.25)
    q3 = pd_series.quantile(.75)
    iqr = q3 - q1
    upper = q3 + iqr*1.5
    lower = q1 - iqr*1.5
    judge = list(pd_series.apply(lambda x:lower < x < upper))

    new_index = []
    #print(len(judge))
    for i in range(len(judge)):
        #print(f"#{i:3} --> {judge[i]}")
        if judge[i]==True:
            new_index.append(labels[i])

    return new_index

In [ ]:
model_a = "n1"
model_b = "s1"

#t = np.arange(0, 10.0, 0.001)  # if test-data is ms2a
t = np.arange(0, 20.0, 0.001)  # if test-data is ms1a
labels = [f"#{i}" for i in range(101)]

df_loss_a = pd.read_csv(f"{work_path}/vrf_{model_a}_loss.csv")
df_resp_a = pd.read_csv(f"{work_path}/vrf_{model_a}_resp.csv")

df_loss_b = pd.read_csv(f"{work_path}/vrf_{model_b}_loss.csv")
df_resp_b = pd.read_csv(f"{work_path}/vrf_{model_b}_resp.csv")

df_error_a = pd.DataFrame({f"#{i}": test_y[:,0] - df_resp_a[labels[i]].values for i in range(len(labels))})
df_error_b = pd.DataFrame({f"#{i}": test_y[:,0] - df_resp_b[labels[i]].values for i in range(len(labels))})


In [ ]:
filt_index_a = filter(pd.Series(df_loss_a[labels].tail(1).values[0]), labels)
filt_index_b = filter(pd.Series(df_loss_b[labels].tail(1).values[0]), labels)

df_loss_af = df_loss_a[filt_index_a]
df_resp_af = df_resp_a[filt_index_a].set_index(t)
df_error_af = df_error_a[filt_index_a].set_index(t)

df_loss_bf = df_loss_b[filt_index_b]
df_resp_bf = df_resp_b[filt_index_b].set_index(t)
df_error_bf = df_error_b[filt_index_b].set_index(t)


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4), dpi=200)
df_loss_af.mean(axis=1).plot(ax=ax[0])
df_loss_bf.mean(axis=1).plot(ax=ax[0])
ax[0].grid()
ax[0].set_ylabel("learning loss (mean)")
ax[0].set_xlabel("epochs")

df_loss_af.mean(axis=1).plot(ax=ax[1])
df_loss_bf.mean(axis=1).plot(ax=ax[1])
ax[1].set_xlim(100, 200)
ax[1].set_ylim(0.0, 0.0004)
ax[1].grid()
ax[1].set_xlabel("epochs")
ax[1].legend(["Conventional Network", "Proposed Network"])

In [ ]:
error_result = pd.DataFrame({
    "Conv."        : pd.Series(np.sqrt(np.square(df_error_a[label].values).mean()) for label in labels),
    "Prop."        : pd.Series(np.sqrt(np.square(df_error_b[label].values).mean()) for label in labels),
    "Conv.(filter)": pd.Series(np.sqrt(np.square(df_error_af[label].values).mean()) for label in filt_index_a),
    "Prop.(filter)": pd.Series(np.sqrt(np.square(df_error_bf[label].values).mean()) for label in filt_index_b)})

fig, ax = plt.subplots(1,2,figsize=(12,4), dpi=200)

error_result.plot.box(ax=ax[0])
error_result.plot.box(ax=ax[1], y=["Conv.(filter)", "Prop.(filter)"])
ax[0].set_ylabel("Estimated error (RMSE)")

In [ ]:
error_result.describe().loc[["count", "mean", "std"]]

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(5, 5), dpi=200)

ax[0].plot(t, test_y[:, 0], "r:")
df_resp_af.mean(axis=1).plot(ax=ax[0])
df_resp_bf.mean(axis=1).plot(ax=ax[0])
ax[0].set_ylabel("Estimated response")
ax[0].legend(["Reference", "Conventional Network", "Proposed Network"], loc=1)
ax[0].grid()

df_error_af.mean(axis=1).plot(ax=ax[1])
df_error_bf.mean(axis=1).plot(ax=ax[1])
ax[1].set_ylabel("Estimated error")
ax[1].set_xlabel("Time [sec]")
ax[1].grid()

In [ ]:
rmse_af = np.array([np.sqrt(np.square(df_error_af[label].values).mean()) for label in filt_index_a]).mean()
rmse_bf = np.array([np.sqrt(np.square(df_error_bf[label].values).mean()) for label in filt_index_b]).mean()

print(f"{rmse_af:.4e}, {rmse_bf:.4e}")

In [ ]:
(rmse_af - rmse_bf)/rmse_af * 100